## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

##Cleaning the Data

In [2]:
df_new = pd.read_csv('data.csv',encoding="ISO-8859-1",dtype={'CustomerID': str,'InvoiceID': str})

In [3]:
df_new.dropna()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680,France


In [4]:
df_new.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [5]:
df_new = df_new[df_new['Quantity'] >= 0]
df_new = df_new[df_new['UnitPrice'] >= 0]

In [6]:
df_new.isnull().sum()

InvoiceNo           0
StockCode           0
Description       592
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     133359
Country             0
dtype: int64

In [7]:
df_new = df_new.dropna()

In [8]:
#last date available in our dataset
date_string = df_new['InvoiceDate'].max()
datetime_object = datetime.strptime(date_string, "%m/%d/%Y %H:%M")
now = datetime_object.date()

In [9]:
now

datetime.date(2011, 9, 9)

In [10]:
df_new['date'] = pd.DatetimeIndex(df_new['InvoiceDate']).date

In [11]:
now = df_new['date'].max()

In [12]:
recency_df = df_new.groupby(by='Description', as_index=False)['date'].max()
recency_df.columns = ['Description','LastPurshaceDate']
recency_df.head()

,Description,LastPurshaceDate
0,4 PURPLE FLOCK DINNER CANDLES,2011-12-08
1,50'S CHRISTMAS GIFT BAG LARGE,2011-12-09
2,DOLLY GIRL BEAKER,2011-12-08
3,I LOVE LONDON MINI BACKPACK,2011-12-08
4,I LOVE LONDON MINI RUCKSACK,2011-10-06


In [13]:
#calculate recency
recency_df['Recency'] = recency_df['LastPurshaceDate'].apply(lambda x: (now - x).days)

In [14]:
recency_df.sort_values(by = 'Recency')

,Description,LastPurshaceDate,Recency
2718,RED RETRO KITCHEN WALL CLOCK,2011-12-09,0
1943,LUNCH BAG RED RETROSPOT,2011-12-09,0
1166,FAIRY TALE COTTAGE NIGHT LIGHT,2011-12-09,0
663,CHARLIE + LOLA RED HOT WATER BOTTLE,2011-12-09,0
2302,PANTRY MAGNETIC SHOPPING LIST,2011-12-09,0
...,...,...,...
2501,PINK MARSHMALLOW SCARF KNITTING KIT,2010-12-01,373
1400,GIRLY PINK TOOL SET,2010-12-01,373
2619,POTTING SHED SOW 'N' GROW SET,2010-12-01,373
2762,RED WHITE SCARF HOT WATER BOTTLE,2010-12-01,373


In [15]:
recency_df.drop('LastPurshaceDate',axis=1,inplace=True)

In [16]:
# drop duplicates
df_new_copy = df_new
df_new_copy.drop_duplicates(subset=['InvoiceNo', 'Description'], keep="first", inplace=True)
#calculate frequency of purchases
frequency_df = df_new_copy.groupby(by=['Description'], as_index=False)['InvoiceNo'].count()
frequency_df.columns = ['Description','Frequency']
frequency_df.head()

,Description,Frequency
0,4 PURPLE FLOCK DINNER CANDLES,36
1,50'S CHRISTMAS GIFT BAG LARGE,109
2,DOLLY GIRL BEAKER,133
3,I LOVE LONDON MINI BACKPACK,68
4,I LOVE LONDON MINI RUCKSACK,1


###Monetory

In [17]:
#create column total cost
df_new['TotalCost'] = df_new['Quantity'] *df_new['UnitPrice']

In [18]:
monetary_df = df_new.groupby(by='Description',as_index=False).agg({'TotalCost': 'sum'})
monetary_df.columns = ['Description','Monetary']
monetary_df.head()

,Description,Monetary
0,4 PURPLE FLOCK DINNER CANDLES,255.46
1,50'S CHRISTMAS GIFT BAG LARGE,2272.25
2,DOLLY GIRL BEAKER,2738.25
3,I LOVE LONDON MINI BACKPACK,1433.25
4,I LOVE LONDON MINI RUCKSACK,4.15


In [19]:
#merge recency dataframe with frequency dataframe
temp_df = recency_df.merge(frequency_df,on='Description')
temp_df.head()

,Description,Recency,Frequency
0,4 PURPLE FLOCK DINNER CANDLES,1,36
1,50'S CHRISTMAS GIFT BAG LARGE,0,109
2,DOLLY GIRL BEAKER,1,133
3,I LOVE LONDON MINI BACKPACK,1,68
4,I LOVE LONDON MINI RUCKSACK,64,1


In [20]:
#merge with monetary dataframe to get a table with the 3 columns
rfm_df = temp_df.merge(monetary_df,on='Description')
#use CustomerID as index
rfm_df.set_index('Description',inplace=True)
#check the head
rfm_df.head()

,Recency,Frequency,Monetary
Description,,,
4 PURPLE FLOCK DINNER CANDLES,1,36,255.46
50'S CHRISTMAS GIFT BAG LARGE,0,109,2272.25
DOLLY GIRL BEAKER,1,133,2738.25
I LOVE LONDON MINI BACKPACK,1,68,1433.25
I LOVE LONDON MINI RUCKSACK,64,1,4.15


##Quartiles

In [21]:
quantiles = rfm_df.quantile(q=[0.25,0.5,0.75])
quantiles

,Recency,Frequency,Monetary
0.25,1.0,10.0,114.15
0.50,4.0,43.0,576.90
0.75,53.0,125.0,1968.00


In [22]:
quantiles.to_dict()

{'Recency': {0.25: 1.0, 0.5: 4.0, 0.75: 53.0},
 'Frequency': {0.25: 10.0, 0.5: 43.0, 0.75: 125.0},
 'Monetary': {0.25: 114.15, 0.5: 576.9, 0.75: 1967.9999999999998}}

##Creation of RFM Segments

In [23]:
# Arguments (x = value, p = recency, monetary_value, frequency, d = quartiles dict)
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]:
        return 2
    else:
        return 1
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]:
        return 3
    else:
        return 4

In [24]:
#create rfm segmentation table
rfm_segmentation = rfm_df
rfm_segmentation['R_Quartile'] = rfm_segmentation['Recency'].apply(RScore, args=('Recency',quantiles,))
rfm_segmentation['F_Quartile'] = rfm_segmentation['Frequency'].apply(FMScore, args=('Frequency',quantiles,))
rfm_segmentation['M_Quartile'] = rfm_segmentation['Monetary'].apply(FMScore, args=('Monetary',quantiles,))

In [25]:
rfm_segmentation.head()

,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile
Description,,,,,,
4 PURPLE FLOCK DINNER CANDLES,1,36,255.46,4,2,2
50'S CHRISTMAS GIFT BAG LARGE,0,109,2272.25,4,3,4
DOLLY GIRL BEAKER,1,133,2738.25,4,4,4
I LOVE LONDON MINI BACKPACK,1,68,1433.25,4,3,3
I LOVE LONDON MINI RUCKSACK,64,1,4.15,1,1,1


In [26]:
rfm_segmentation['RFMScore'] = rfm_segmentation.R_Quartile.map(str) \
                            + rfm_segmentation.F_Quartile.map(str) \
                            + rfm_segmentation.M_Quartile.map(str)
rfm_segmentation.head()

,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile,RFMScore
Description,,,,,,,
4 PURPLE FLOCK DINNER CANDLES,1,36,255.46,4,2,2,422
50'S CHRISTMAS GIFT BAG LARGE,0,109,2272.25,4,3,4,434
DOLLY GIRL BEAKER,1,133,2738.25,4,4,4,444
I LOVE LONDON MINI BACKPACK,1,68,1433.25,4,3,3,433
I LOVE LONDON MINI RUCKSACK,64,1,4.15,1,1,1,111


In [27]:
def classify_customer(df):
    if (df['RFMScore'] == 444):
        return 'Best customer Choice'
    elif (df['RFMScore'] == 244):
            return "Almost Lost Customer Purchase"
    elif (df['RFMScore'] == 144):
            return "Lost Customer Purchase"
    elif (df['RFMScore'] == 111):
            return "Lost Low Spending Customer Purchase"
    else:
        return "Others"

In [28]:
rfm_segmentation['RFMScore'] = rfm_segmentation['RFMScore'].apply(pd.to_numeric)

In [29]:
rfm_segmentation['Product_Segmentation'] = rfm_segmentation.apply(classify_customer,axis=1)

In [30]:
def classify_customer_2(df):
    if (df['F_Quartile'] == 4):
        return 'Loyal Customers Choice'
    elif (df['M_Quartile'] == 4):
            return "Big Spenders Choice"
    elif (df['R_Quartile'] == 4):
      return "New Customers Choice"
    else:
      return "NA"

In [31]:
rfm_segmentation['Prod_Seg_RFM'] = rfm_segmentation.apply(classify_customer_2,axis=1)

In [32]:
rfm_segmentation.shape

(3877, 9)

In [33]:
rfm_segmentation.head()

,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile,RFMScore,Product_Segmentation,Prod_Seg_RFM
Description,,,,,,,,,
4 PURPLE FLOCK DINNER CANDLES,1,36,255.46,4,2,2,422,Others,New Customers Choice
50'S CHRISTMAS GIFT BAG LARGE,0,109,2272.25,4,3,4,434,Others,Big Spenders Choice
DOLLY GIRL BEAKER,1,133,2738.25,4,4,4,444,Best customer Choice,Loyal Customers Choice
I LOVE LONDON MINI BACKPACK,1,68,1433.25,4,3,3,433,Others,New Customers Choice
I LOVE LONDON MINI RUCKSACK,64,1,4.15,1,1,1,111,Lost Low Spending Customer Purchase,NA


In [34]:
rfm_segmentation[rfm_segmentation.Product_Segmentation == 'Best customer Choice' ]

,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile,RFMScore,Product_Segmentation,Prod_Seg_RFM
Description,,,,,,,,,
DOLLY GIRL BEAKER,1,133,2738.25,4,4,4,444,Best customer Choice,Loyal Customers Choice
SPACEBOY BABY GIFT SET,1,142,7048.55,4,4,4,444,Best customer Choice,Loyal Customers Choice
12 MESSAGE CARDS WITH ENVELOPES,0,187,2547.30,4,4,4,444,Best customer Choice,Loyal Customers Choice
12 PENCIL SMALL TUBE WOODLAND,1,276,2452.35,4,4,4,444,Best customer Choice,Loyal Customers Choice
12 PENCILS SMALL TUBE RED RETROSPOT,1,307,3979.10,4,4,4,444,Best customer Choice,Loyal Customers Choice
...,...,...,...,...,...,...,...,...,...
YOU'RE CONFUSING ME METAL SIGN,0,342,6708.49,4,4,4,444,Best customer Choice,Loyal Customers Choice
ZINC FOLKART SLEIGH BELLS,1,339,7612.72,4,4,4,444,Best customer Choice,Loyal Customers Choice
ZINC METAL HEART DECORATION,1,476,7053.68,4,4,4,444,Best customer Choice,Loyal Customers Choice


##Second Method

Fetch the Customer ID --> Identify its transactions --> Create top 3 Recent Purchases --> Top 3 Frequently Purchased --> Top 3 Expensive Products Purchased

In [35]:
df_new.shape

(387772, 10)

In [36]:
df_new.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date,TotalCost
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom,2010-12-01,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom,2010-12-01,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom,2010-12-01,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom,2010-12-01,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom,2010-12-01,20.34


In [37]:
df_new['CustomerID'] = df_new['CustomerID'].apply(pd.to_numeric)

In [38]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 387772 entries, 0 to 541908
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    387772 non-null  object 
 1   StockCode    387772 non-null  object 
 2   Description  387772 non-null  object 
 3   Quantity     387772 non-null  int64  
 4   InvoiceDate  387772 non-null  object 
 5   UnitPrice    387772 non-null  float64
 6   CustomerID   387772 non-null  int64  
 7   Country      387772 non-null  object 
 8   date         387772 non-null  object 
 9   TotalCost    387772 non-null  float64
dtypes: float64(2), int64(2), object(6)
memory usage: 32.5+ MB


In [40]:
def get_customer_data(customerID, df):
  df_transaction = df_new[df_new['CustomerID'] ==  customerID]
  df_transaction = df_transaction.sort_values(by='date', ascending=False)

  # Get the most recently purchased 3 products
  recent_products = df_transaction.head(3)['Description'].tolist()

 #get the most frequently purchased products
  product_totals = df_transaction.groupby(['Description'])['Quantity'].sum()

  sorted_product_totals = product_totals.sort_values(ascending=False)

  # Get the top 3 products purchased
  top_3_products = sorted_product_totals.head(3).reset_index().values.tolist()

  sorted_price_totals = df_new.sort_values(by='TotalCost')
  top_3_M_products = sorted_price_totals.head(3)['Description'].tolist()

  return recent_products ,top_3_products, top_3_M_products

In [41]:
recent_products =[]
top_3_products = []
top_3_M_products=[]

In [42]:
recent_products,top_3_products,top_3_M_products = get_customer_data(17850, df_new)
print("Recent Products", recent_products)
print("Frequent Products", top_3_products)
print("Monetary Products", top_3_M_products)

Recent Products ['WHITE METAL LANTERN', 'CREAM CUPID HEARTS COAT HANGER', 'WHITE HANGING HEART T-LIGHT HOLDER']
Frequent Products [['WHITE METAL LANTERN', 122], ['WHITE HANGING HEART T-LIGHT HOLDER', 122], ['KNITTED UNION FLAG HOT WATER BOTTLE', 110]]
Monetary Products ['PASTEL COLOUR HONEYCOMB FAN', 'SET OF 6 SOLDIER SKITTLES', 'CHRISTMAS PUDDING TRINKET POT ']
